## What is Theano?

- Python library developed by machine learning researchers at the University of Montreal
- define, optimize, and evaluate mathematical expressions involving multi-dimensional arrays efficiently
- tight integration with NumPy - use numpy.ndarray in Theano-compiled functions
- **efficient symbolic differentiation** - Theano does your derivatives for function with one or many inputs
- supports both CPU and GPU
- uses fast back-ends (CUDA, BLAS, dynamically generated C code)
- supports CuDNN v3

In [1]:
import theano
from theano import tensor
import numpy
numpy.random.seed(1234)
theano.config.floatX = 'float64'

# allows declaration of symbolic variables
x = tensor.vector('x')
W = tensor.matrix('W')
b = tensor.vector('b')

# function syntax is similar to NumPy
xW  = tensor.dot(x, W)
out = tensor.nnet.sigmoid(xW + b)

# create a function variable which you can call to get output on given input
f = theano.function(inputs=[x, W, b], outputs=out)

# evaluate the above function for some input
x_val = numpy.random.rand(4)           # 1 x 4
W_val = numpy.random.randn(4, 3)       # 4 x 3
b_val = numpy.random.rand(3)           # 1 x 3

print 'f:', f(x_val, W_val, b_val)     # 1 x 3

f: [ 0.30044614  0.77454353  0.3901409 ]


### Strong Typing
- all Theano variables have a type
- Examples
  - TensorType for NumPy ndarrays
  - CudaNdarrayType for CUDA arrays
  - Sparse for scipy.sparse matrices

In [2]:
# create theano variables using theano.shared()
theano.config.floatX = 'float32'
tvar = theano.shared(numpy.array([[1, 2], [3, 4]], dtype=theano.config.floatX))
print tvar.type()

<TensorType(float32, matrix)>


In [3]:
# Set and get variable value
tvar.set_value(numpy.array([[3, 4], [2, 1]], dtype=theano.config.floatX))
print tvar.get_value()

[[ 3.  4.]
 [ 2.  1.]]


In [4]:
print tvar

<TensorType(float32, matrix)>


In [5]:
a = theano.shared(1)
b = tensor.scalar()
add = a + b
adder = theano.function(inputs=[b], outputs=add)
print adder(3)

4.0


### Exercise caution when using Python keywords
Theano does not redefine Python keywords for Theano shared variables

In [6]:
var = 0
if var:
    print 'True'
else:
    print 'False'

False


In [7]:
var = theano.shared(var)
if var:
    print 'True'
else:
    print 'False'

True


In [8]:
if var.get_value():
    print 'True'
else:
    print 'False'

False


- If you need an if-else expr in your symbolic graph use `theano.ifelse.ifelse()`
- Similary `for i in var:` will not work; Use `theano.scan()` for looping

### Using the GPU

- default device can be specified in the .theanorc file
- use float32 when using most GPUs
- floatX can be specified as a command line flag

For CPU:
```
THEANO_FLAGS=device=cpu,floatX=float32 python your_script.py
```
For GPU:
```
THEANO_FLAGS=device=gpu0,floatX=float32 python your_script.py
```
To obtain the current config parameters:
```
print theano.config.device
print theano.config.floatX
```

### Gradients
#### Theano has graph-based automatic symbolic differentiation

In [9]:
x = tensor.vector('x')
W = tensor.matrix('W')
b = tensor.vector('b')
y = tensor.vector('y')

xW = tensor.dot(x,W)
out = tensor.nnet.sigmoid(xW + b)

C = ((out - y) ** 2).sum()

# compute gradients w.r.t. parameters
dC_dW, dC_db = theano.grad(C, wrt=[W, b])
cost_and_grads = theano.function(inputs=[x, W, b, y], outputs=[C, dC_dW, dC_db])

In [10]:
x_val = numpy.random.rand(4).astype('float32')
W_val = numpy.random.randn(4, 3).astype('float32')
b_val = numpy.random.rand(3).astype('float32')
y_val = numpy.random.uniform(size=3).astype('float32')

C_val, dC_dW_val, dC_db_val = cost_and_grads(x_val, W_val, b_val, y_val)
print 'C:\n',C_val
print 'dC/dW:\n',dC_dW_val
print 'dC/db:\n',dC_db_val

C:
0.968002200127
dC/dW:
[[ 0.14226542  0.12770246  0.15071078]
 [ 0.01742641  0.01564256  0.0184609 ]
 [ 0.08526361  0.07653563  0.09032514]
 [ 0.21572049  0.19363831  0.22852637]]
dC/db:
[ 0.23117696  0.20751259  0.24490039]


## Looping - theano.scan()

### `sequences` - variables that `scan()` should iterate over as it loops

Suppose we wish to compute the element-wise product of two vectors

In [11]:
vector1 = tensor.vector('vector1')
vector2 = tensor.vector('vector2')

In [12]:
output, updates = theano.scan(fn=lambda a, b : a * b,
                              sequences=[vector1, vector2])

The first iteration will take as input the first element of every sequence, the second iteration will take as input the second element of every sequence, etc. These individual element have will have one less dimension than the original sequences. For example, for a matrix sequence, the individual elements will be vectors.

The parameter fn receives a function or lambda expression that expresses the computation to do at every iteration. It operates on the symbolic inputs to produce symbolic outputs. It will only ever be called once, to assemble the Theano graph used by Scan at every the iterations.

Since we wish to iterate over both `vector1` and `vector2` simultaneously, we provide them as sequences. This means that every iteration will operate on two inputs: an element from `vector1` and the corresponding element from `vector2`.

Because what we want is the elementwise product between the vectors, we provide a lambda expression that, given an element `a` from `vector1` and an element `b` from `vector2` computes and return the product.

In [13]:
f = theano.function(inputs=[vector1, vector2],
                    outputs=output,
                    updates=updates)

Calling `scan()`, we see that it returns two outputs.
The first output **`output`** contains the outputs of fn from every timestep concatenated into a tensor. In our case, the output of a single timestep is a scalar so output is a vector where output[i] is the output of the i-th iteration.
The second output **`updates`** details if and how the execution of the `scan()` updates any shared variable in the graph. It should be provided as an argument when compiling the Theano function.

If updates is omitted, the state of any shared variables modified by `scan()` will not be updated properly. **Always provide updates when compiling your Theano function**.

In [14]:
vector1_value = numpy.arange(0, 5).astype(theano.config.floatX) # [0,1,2,3,4]
vector2_value = numpy.arange(1, 6).astype(theano.config.floatX) # [1,2,3,4,5]
print(f(vector1_value, vector2_value))

[  0.   2.   6.  12.  20.]


An interesting thing is that we never explicitly told `scan()` how many iteration it needed to run. It was automatically inferred; when given sequences, `scan()` will run as many iterations as the length of the shortest sequence.

### `non_sequences` - variables required to be available 'as is' in each iteration of `scan()`
- We do not want `scan()` to iterate over them and give only part of them at every iteration. We want the complete variable.

In [15]:
X = tensor.matrix('X')
W = tensor.matrix('W')
b = tensor.vector('b')

To have the full weight matrix `W` and the full bias vector `b` available at every iteration, we use the argument `non_sequences`. 
- Every non-sequence is passed as input to every iteration
- `fn` will have access to both `sequences` and `non_sequences`

In [16]:
def step(v, W, b):
    return tensor.dot(v, W) + b

output, updates = theano.scan(fn=step,
                              sequences=[X],
                              non_sequences=[W, b])

In [17]:
f = theano.function(inputs=[X, W, b],
                    outputs=output,
                    updates=updates)

/home/shikhar/.local/lib/python2.7/site-packages/theano/scan_module/scan_perform_ext.py:135: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  from scan_perform.scan_perform import *


The inputs that correspond to the non-sequences are always last and in the same order as the non-sequences are provided to `scan()`.
- `v` : individual element of the sequence `X`
- `W` and `b` : non-sequences `W` and `b`, respectively

In [18]:
X_value = numpy.arange(-3, 3).reshape(3, 2).astype(theano.config.floatX)
W_value = numpy.eye(2).astype(theano.config.floatX)
b_value = numpy.arange(2).astype(theano.config.floatX)
print 'X_value:\n', X_value
print 'W_value:\n', W_value
print 'b_value:\n', b_value
print 'f:\n',(f(X_value, W_value, b_value))

X_value:
[[-3. -2.]
 [-1.  0.]
 [ 1.  2.]]
W_value:
[[ 1.  0.]
 [ 0.  1.]]
b_value:
[ 0.  1.]
f:
[[-3. -1.]
 [-1.  1.]
 [ 1.  3.]]


### theano.scan() - reusing outputs from the previous iterations
In this example, we will use Scan to compute a cumulative sum over the first dimension of a matrix M. This means that the output will be a matrix S in which the first row will be equal to the first row of M, the second row will be equal to the sum of the two first rows of M, and so on.

Another way to express this, which is the way we will implement here, is that S[t]=S[t−1]+M[t]. Implementing this with Scan would involve iterating over the rows of the matrix M and, at every iteration, reuse the cumulative row that was output at the previous iteration and return the sum of it and the current row of M.

If we assume for a moment that we can get Scan to provide the output value from the previous iteration as an input for every iteration, implementing a step function is simple:

In [19]:
def step(m_row, cumulative_sum):
    return m_row + cumulative_sum

The trick part is informing Scan that our step function expects as input the output of a previous iteration. To achieve this, we need to use a new parameter of the scan() function: outputs_info. This parameter is used to tell Scan how we intend to use each of the outputs that are computed at each iteration.

- For a non-recurrent output (like in every example before this one), the element should be None.
- For a simple recurrent output (iteration t depends on the value at iteration t−1), the element must be a tensor. Scan will interpret it as being an initial state for a recurrent output and give it as input to the first iteration, pretending it is the output value from a previous iteration. For subsequent iterations, Scan will automatically handle giving the previous output value as an input.

The step() function needs to expect one additional input for each simple recurrent output. These inputs correspond to outputs from previous iteration and are always after the inputs that correspond to sequences but before those that correspond to non-sequences. The are received by the step() function in the order in which the recurrent outputs are declared in the outputs_info sequence.

In [20]:
M = tensor.matrix('X')
s = tensor.vector('s') # Initial value for the cumulative sum

output, updates = theano.scan(fn=step,
                              sequences=[M],
                              outputs_info=[s])

We can now compile and test the Theano function :

In [21]:
f = theano.function(inputs=[M, s],
                    outputs=output,
                    updates=updates)

M_value = numpy.arange(9).reshape(3, 3).astype(theano.config.floatX)
s_value = numpy.zeros((3, ), dtype=theano.config.floatX)
print(f(M_value, s_value))

[[  0.   1.   2.]
 [  3.   5.   7.]
 [  9.  12.  15.]]


### theano.scan() - reusing outputs from multiple past iterations
The Fibonacci sequence : 1, 1, 2, 3, 5, 8, 13, ...
F[n] = F[n-1] + F[n-2]

In [ ]:
def step(f_minus2, f_minus1):
    new_f = f_minus2 + f_minus1
    return new_f

- for non-recurrent outputs, the value is None
- for simple recurrent outputs, the value is a single initial state
- for general recurrent outputs, where iteration t may depend on multiple past values, the value is a dictionary
That dictionary has two values:
- **taps** : list declaring which previous values of that output every iteration will need. [-2, -1] would mean every iteration should take as input the last 2 values of that output.
- **initial** : tensor of initial values. If every initial value has n dimensions, initial will be a single tensor of n+1 dimensions with as many initial values as the oldest requested tap. In the case of the Fibonacci sequence, the individual initial values are scalars so the initial will be a vector.

In [22]:
f_init = tensor.fvector()
outputs_info = [dict(initial=f_init, taps=[-2, -1])]

Now that we have no sequence, we need to explicitly tell Scan how many iterations to run using the n_step parameter. The value can be real or symbolic.

In [23]:
output, updates = theano.scan(fn=step,
                              outputs_info=outputs_info,
                              n_steps=10)

next_fibonacci_terms = output

Let's compile our Theano function which will take a vector of consecutive values from the Fibonacci sequence and compute the next 10 values :

In [24]:
f = theano.function(inputs=[f_init],
                    outputs=next_fibonacci_terms,
                    updates=updates)

out = f([1, 1])
print(out)

[   2.    3.    5.    8.   13.   21.   34.   55.   89.  144.]


### Psuedocode for softmax regression

In [ ]:
def sgd(lr, tparams, grads, inp, cost):
    gshared = [theano.shared(p.get_value() * numpy.float32(0.), name='%s_grad'%k) for k, p in tparams.iteritems()]
    gsup = [(gs, g) for gs, g in zip(gshared, grads)]

    f_grad_shared = theano.function(inp, cost, updates=gsup, profile=False)

    pup = [(p, p - lr * g) for p, g in zip(itemlist(tparams), gshared)]
    f_update = theano.function([lr], [], updates=pup, profile=False)

    return f_grad_shared, f_update

def build_model(tparams, options):
    trng = RandomStreams(1234)
    use_noise = theano.shared(numpy.float32(0.))

    x = tensor.matrix('x', dtype='float32')
    y = tensor.vector('y', dtype='int64')
    W = tparams['W']
    b = tparams['b']

    sm    = tensor.nnet.softmax(tensor.dot(x,W)+b)
    cost  = -(tensor.log(sm[tensor.arange(y.shape[0]), y]+1e-8)).mean()
    wdec  = W*W
    cost  = cost + options['decay_c']*wdec.sum().sum()/2
    on    = tensor.argmax(sm,axis=1)
    pred  = tensor.eq(on,y)
    pred  = pred.reshape([pred.shape[0],1])
    
    return trng, use_noise, x, y, pred, cost

..............
def main():
    trng, use_noise, x, y, pred, cost = build_model(tparams, model_options)
    grads                    = tensor.grad(cost, wrt=itemlist(tparams))
    lr                       = tensor.scalar(name='lr')
    f_grad_shared, f_update  = eval('sgd')(lr, tparams, grads, [x,y], cost)
    f_preds                  = theano.function(inputs=[x,y], outputs=pred)

    for epochidx in xrange(max_epochs):
        for batchidx in iterator:
            x, y, n_ex = getBatch(iterator)
            cost = f_grad_shared(x, y)       # forward prop
            f_update(lrate)                  # back prop

## Frameworks
**Lasagne** - lightweight library to build and train neural networks in Theano

**blocks** - a Theano framework for building and training neural networks

**keras**

## Resources
http://deeplearning.net/software/theano/tutorial/index.html#tutorial
https://github.com/mila-udem/summerschool2015

### talk about googlenet implementations

In [ ]:
from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet

net1 = NeuralNet(
    layers=[  # three layers: one hidden layer
        ('input', layers.InputLayer),
        ('hidden', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],
    # layer parameters:
    input_shape=(None, 9216),  # 96x96 input pixels per batch
    hidden_num_units=100,  # number of units in hidden layer
    output_nonlinearity=None,  # output layer uses identity function
    output_num_units=30,  # 30 target values

    # optimization method:
    update=nesterov_momentum,
    update_learning_rate=0.01,
    update_momentum=0.9,

    regression=True,  # flag to indicate we're dealing with regression problem
    max_epochs=400,  # we want to train this many epochs
    verbose=1,
    )

X, y = load()
net1.fit(X, y)

In [ ]:
net2 = NeuralNet(
    layers=[
        ('input', layers.InputLayer),
        ('conv1', layers.Conv2DLayer),
        ('pool1', layers.MaxPool2DLayer),
        ('conv2', layers.Conv2DLayer),
        ('pool2', layers.MaxPool2DLayer),
        ('conv3', layers.Conv2DLayer),
        ('pool3', layers.MaxPool2DLayer),
        ('hidden4', layers.DenseLayer),
        ('hidden5', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],
    input_shape=(None, 1, 96, 96),
    conv1_num_filters=32, conv1_filter_size=(3, 3), pool1_pool_size=(2, 2),
    conv2_num_filters=64, conv2_filter_size=(2, 2), pool2_pool_size=(2, 2),
    conv3_num_filters=128, conv3_filter_size=(2, 2), pool3_pool_size=(2, 2),
    hidden4_num_units=500, hidden5_num_units=500,
    output_num_units=30, output_nonlinearity=None,

    update_learning_rate=0.01,
    update_momentum=0.9,

    regression=True,
    max_epochs=1000,
    verbose=1,
    )

X, y = load2d()  # load 2-d data
net2.fit(X, y)

### Debugging